https://drive.google.com/file/d/164mK6i85AthlqvijDIPIHfs71s8gZHdt/view?usp=sharing

Загружаем архив с данными

In [ ]:
! gdown --id 164mK6i85AthlqvijDIPIHfs71s8gZHdt

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=164mK6i85AthlqvijDIPIHfs71s8gZHdt
To: /content/alpha_competition.zip
100% 35.2M/35.2M [00:00<00:00, 75.2MB/s]


Разархивируем данные

In [ ]:
! unzip /content/alpha_competition.zip

Archive:  /content/alpha_competition.zip
  inflating: sample_submission.csv   
  inflating: train_supervised_dataset.csv  
  inflating: test_dataset.csv        
  inflating: train_unsupervised_dataset.csv  


Импортируем библиотеки

In [ ]:
import pandas as pd

Считываем данные с тренировочной выборки

In [ ]:
train_df = pd.read_csv('/content/train_supervised_dataset.csv')
train_df.head()


,id,name,good,brand
0,0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 Бусы для елки шарики_87191,бусы,NaN
2,2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,4,Одежда (вес) 1500,одежда,NaN


Проверяем есть ли пропущенные данные и дубликаты

In [ ]:
train_df.isna().sum()

id          0
name        0
good      819
brand    8505
dtype: int64

In [ ]:
train_df.duplicated().sum()

0

Импортируем библиотеки

In [ ]:
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
import string

Напишем функции, очищающие исходный текст от знаков препинания и убирающие из текста предлоги, союзы и тп.

In [ ]:
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
def clean_data(line):

    tokens = str(line)
    tokens = [re.sub('<[^>]+>', '', token).strip(string.punctuation) for token in tokens.split()]
    line = " ".join(tokens)
    return line

def preprocess_data(text):

    tokens = str(text)
    tokens = [token for token in tokens.split() if token not in russian_stopwords\

              and token.isalpha()\
              and len(token)>=3 ]
    text = " ".join(tokens)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Импортируем необходимые библиотеки

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from imblearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import numpy


Посчитаем количество разных классов в тренировочном датасете

In [ ]:
df_good = pd.DataFrame(train_df['good'].value_counts()).reset_index().rename(columns={"index": "good", "good": "counts"})
df_brand = pd.DataFrame(train_df['brand'].value_counts()).reset_index().rename(columns={"index": "brand", "brand": "counts"})

Отберем только те классы, которых не меньше 4

In [ ]:
df_goods = train_df.loc[train_df["good"].isin(df_good[df_good["counts"] >= 4]["good"])]
df_brands = train_df.loc[train_df["brand"].isin(df_brand[df_brand["counts"] >= 4]["brand"])]
df_brands.shape

(8806, 4)

Удалим пропущенные значения и обработаем тренировочные данные с помощью функций clean_data и preprocess_data

In [ ]:
X0 = df_goods['name'].loc[df_goods['good'].notna()].apply(lambda x: clean_data(x)).apply(lambda x: preprocess_data(x))
X1 = df_brands['name'].loc[df_brands['brand'].notna()].apply(lambda x: clean_data(x)).apply(lambda x: preprocess_data(x))
y0 = df_goods['good'].loc[df_goods['good'].notna()]
y1 = df_brands['brand'].loc[df_brands['brand'].notna()]

Разделим данные на тренировочную и валидационную выборку



In [ ]:
X0_train, X0_val, y0_train, y0_val = train_test_split(X0, y0, stratify=y0,
                                                    test_size=0.15, random_state=42)
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, y1, stratify=y1,
                                                    test_size=0.15, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='weighted')

Соберем пайплан, подберем параметры с помощью GridSearchCV и обучим модель для метки 'good'

In [ ]:
clf = LinearSVC()
parameters = {
'vect__ngram_range': ((1, 1), (1, 2), (1,3)),
}
text_clf_0 = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', OneVsRestClassifier(clf, n_jobs=-1))])
gs_cv = GridSearchCV(estimator=text_clf_0,
                 param_grid=parameters,
                 cv= 4,
                 verbose=1,
                 scoring=f1)
gs_cv.fit(X0_train, y0_train)
y0_pred = gs_cv.best_estimator_.predict(X0_val)
print('Score:', 100*f1_score(y0_val, y0_pred,  average='weighted'))

Fitting 4 folds for each of 3 candidates, totalling 12 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(


Score: 90.76752712018776


Соберем пайплан, подберем параметры с помощью GridSearchCV и обучим модель для метки 'brand'

In [ ]:
text_clf_1 = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', OneVsRestClassifier(clf, n_jobs=-1))])

gs_cv_1 = GridSearchCV(estimator=text_clf_0,
                 param_grid=parameters,
                 cv= 4,
                 verbose=1,
                 scoring=f1)
gs_cv_1.fit(X1_train, y1_train)
y1_pred = gs_cv_1.best_estimator_.predict(X1_val)
print('Score:', 100*f1_score(y1_val, y1_pred,  average='weighted'))

Fitting 4 folds for each of 3 candidates, totalling 12 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(


Score: 87.88366064163905


Считаем тестовые данные

In [ ]:
data_test = pd.read_csv('/content/test_dataset.csv')
data_test.head()

,id,name
0,0,"469-210 ЕРМАК Клей универсальный, 15мл, блистер"
1,1,Торт СЛАДУШКА Зимняя вишня 700г
2,2,"Смеситель ""CALORIE"" 1023 А06 д/кухни"
3,3,Лимон 50гр БАР
4,4,"Коньяк САРАДЖИШВИЛИ 5 лет 0,5л Грузия"


Обработаем тестовые данные с помощью функций clean_data и preprocess_data

In [ ]:
X_test = data_test['name'].apply(lambda x: clean_data(x)).apply(lambda x: preprocess_data(x))

Предскажем метки 'good' и 'brand'

In [ ]:
data_test['good'] = gs_cv.best_estimator_.predict(X_test)
data_test['brand'] = gs_cv_1.best_estimator_.predict(X_test)

Запишем решение в файл 'submission.csv'

In [ ]:
data_test[['id', 'good', 'brand']].to_csv('submission.csv')